In [ ]:
import boto3
import pandas as pd
from pprint import pprint
import json

In [ ]:
secrets = pd.read_json("../secrets/credentials.json")
creds = {"aws_access_key_id": secrets["aws"]["key_id"],
       "aws_secret_access_key": secrets["aws"]["key"]
}

client = boto3.client("kinesis",**creds)

stream_name = "finance"

In [ ]:
stream = client.describe_stream(StreamName=stream_name)
pprint(stream)

In [ ]:
def get_shard_iterator(shard_id):
    shard_iterator = client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType="TRIM_HORIZON"
    )
    return shard_iterator["ShardIterator"]
    
def get_records(shard_id):
    iterator = get_shard_iterator(shard_id)
    records = client.get_records(
        ShardIterator=iterator,
        Limit=5000
    )
    df = pd.DataFrame(records["Records"])
    return df

In [ ]:
shards = stream['StreamDescription']['Shards']
shard_ids = map(lambda s: s["ShardId"], shards)
shard_data = map(lambda s: get_records(s), shard_ids)

df = pd.concat(shard_data)
df.groupby("PartitionKey").count()